In [10]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as plt

test_data = datasets.MNIST(root="data", train=False, download=True, transform=ToTensor())
train_data = datasets.MNIST(root="data", train=True, download=True, transform=ToTensor())

In [11]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [12]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"

class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 768),
        nn.ReLU(),
        nn.Linear(768, 192),
        nn.ReLU(),
        nn.Linear(192, 30),
        nn.ReLU(),
        nn.Linear(30, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = Model().to(device)
print(model)

Model(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=768, bias=True)
    (1): ReLU()
    (2): Linear(in_features=768, out_features=192, bias=True)
    (3): ReLU()
    (4): Linear(in_features=192, out_features=30, bias=True)
    (5): ReLU()
    (6): Linear(in_features=30, out_features=10, bias=True)
  )
)


In [13]:
len(train_dataloader.dataset)

60000

In [25]:
# @title
batch_size = 64

def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    pred = model(X.to(device))
    loss = loss_fn(pred, y.to(device))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * batch_size + len(X)
      print(f'loss: {loss} [{current} / {size}]')

def test_loop(dataloader, model, loss_fn):
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X,y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f'test erorr -- acc: {100*correct}, avg loss: {test_loss}')


learning_rate = 1e-1
epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
  print(f'Epoch: {t+1}\n---------------------------------')
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)


Epoch: 1
---------------------------------
torch.Size([64, 1, 28, 28])
torch.Size([1, 28, 28])
torch.Size([64, 784])


Exception: 